In [20]:
import pandas as pd #importing essential library 
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
#renamed the files to make it more accessible and intuitive.
#reading csv files into pd dataframes for manipulation and data wrangling
cardholder_acc = pd.read_csv('../csvs_datathon/accy_dim.csv')
statement_data = pd.read_csv('../csvs_datathon/statement_fact.csv')
transaction_data = pd.read_csv('../csvs_datathon/transaction_fact.csv')
wrld_transaction_data = pd.read_csv('../csvs_datathon/wrld_stor_tran_fact.csv')
customer_id = pd.read_csv('../csvs_datathon/syf_id.csv')
acc_lvl_features = pd.read_csv('../csvs_datathon/rams_batch_cur.csv')
fraud_claim_case = pd.read_csv('../csvs_datathon/fraud_claim_case.csv')
fraud_claim_tran = pd.read_csv('../csvs_datathon/fraud_claim_tran.csv')

In [ ]:
transaction_data

In [ ]:
transaction_data = transaction_data[transaction_data['transaction_type'] == 'SALE']
# print(transaction_data)
transaction_data.dtypes
# print(transaction_data['transaction_date'].dtype)
transaction_data['transaction_date'] = pd.to_datetime(transaction_data['transaction_date'])
# print(transaction_data['transaction_date'].dtype)
transaction_data['year'] = transaction_data['transaction_date'].dt.year
transaction_data['month'] = transaction_data['transaction_date'].dt.month
transaction_data['day'] = transaction_data['transaction_date'].dt.day
transaction_data['dayofweek'] = transaction_data['transaction_date'].dt.dayofweek
# transaction_data['quarter'] = transaction_data['transaction_date'].dt.quarter
# transaction_data['date'] = transaction_data['transaction_date'].dt.date

transaction_data_sorted_chronological = transaction_data.sort_values(by='transaction_date', ascending=True).reset_index(drop=True)
# transaction_data_sorted
# print(transaction_data['year'].dtype)
transaction_data_sorted_chronological.head(50)
# transaction_data_sorted.tail(50)
# transaction_data_sorted.sample(50)

#filter only aug to mar

transaction_data_last8months = transaction_data_sorted_chronological[(transaction_data_sorted_chronological['transaction_date'] >= '2024-08-01') & (transaction_data_sorted_chronological['transaction_date'] <= '2025-03-24')]
transaction_data_last8months.tail()

In [ ]:
import statsmodels.formula.api as smf
# transaction_data_last8months.isna().sum()
# transaction_type, transaction_code, payment_type, transaction_return_cnt, transaction_sale_cnt, 
# product_amt, product_qty, fcr_flag, fcr_rate_of_exchange, frgn_tran_amt, us_equiv_amt, year, month, dayofweek
results = smf.ols('transaction_amt ~ transaction_type + year + month + dayofweek + us_equiv_amt + frgn_tran_amt + us_equiv_amt + transaction_sale_cnt', data=transaction_data_last8months).fit()
print(results.summary())

In [25]:
# transaction_data_last8months.dtypes
def create_customer_df(account_id):
    df = transaction_data_last8months[transaction_data_last8months['current_account_nbr'] == account_id]
    return df

In [ ]:
# TEST FUNCTIONS

my_df = create_customer_df('Z87LjFfrML6krD0n')
my_df

In [ ]:
# visualizations test

plt.scatter(my_df['transaction_date'], my_df['transaction_amt'])
plt.show()

plt.plot(my_df['transaction_date'], my_df['transaction_amt'])
plt.show()

In [ ]:
transaction_data_last8months['transaction_amt'].describe()

# check dist
plt.figure(figsize=(10, 6))
sns.histplot(transaction_data_last8months['transaction_amt'], bins=50, kde=True)
plt.xlabel('transaction_amt')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#exclude outliers

transaction_data_last8months

mean_amt = transaction_data_last8months['transaction_amt'].mean()
std_amt = transaction_data_last8months['transaction_amt'].std()

print(mean_amt)
print(std_amt)

lower_bound = mean_amt - 3 * std_amt
upper_bound = mean_amt + 3 * std_amt

#filter data
filtered_transaction_data_no_outliers = transaction_data_last8months[(transaction_data_last8months['transaction_amt'] >= lower_bound) &
                                                         (transaction_data_last8months['transaction_amt'] <= upper_bound)]


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(filtered_transaction_data_no_outliers['transaction_amt'], bins=50, kde=True)
plt.xlabel('transaction_amt')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# set index with transaction date
filtered_transaction_data_no_outliers.set_index('transaction_date', inplace=True)
# resample by month aggregate to get the sum
monthly_transaction_data = (filtered_transaction_data_no_outliers.groupby('current_account_nbr').resample('M')['transaction_amt'].sum().reset_index())
monthly_transaction_data.set_index('transaction_date', inplace=True)
monthly_transaction_data.head(50)

In [ ]:
test_transaction_data = monthly_transaction_data[monthly_transaction_data['current_account_nbr'] == 'Z87LjFfrML6krD0n']
test_transaction_data

In [ ]:
adf_result = adfuller(monthly_transaction_data['transaction_amt'])
adf_result

In [34]:

# evaluate how good our model is first / check the mse and r2 values
def evaluate_model(account_id, transaction_data):
    account_data = transaction_data[transaction_data['current_account_nbr'] == account_id]
    account_data = account_data.sort_index()

    # first 6 months
    train_data = account_data.iloc[:-2]  
    # last 2 months
    test_data = account_data.iloc[-2:]  

    model = AutoReg(train_data['transaction_amt'], lags=2)

    predictions = model.fit().predict(start=test_data.index[0], end=test_data.index[-1])

    mse = mean_squared_error(test_data['transaction_amt'], predictions)
    r2 = r2_score(test_data['transaction_amt'], predictions)
    
    print('MSE VALUE:')
    print(mse)
    print('R² VALUE:')
    print(r2)

    return mse, r2

In [35]:

# use past data to predict q4 spending
def predict_q4_spending(account_id, transaction_data):
    account_data = transaction_data[transaction_data['current_account_nbr'] == account_id]
    # print(account_data)
    account_data = account_data.sort_index()
    # print(account_data)
    # lags - determines how far back model will look at to make prediction
    model = AutoReg(account_data['transaction_amt'], lags=2)
    
    start_date = '2025-10-01'
    end_date = '2025-12-31'
    predictions = model.fit().predict(start=start_date, end=end_date)
    # goes 10-31 then 11-30 then 12-31
    # print(predictions)
    return predictions.sum()

In [ ]:
id = 'Z87LjFfrML6krD0n'

mse_and_r2 = evaluate_model(id, monthly_transaction_data)
print(mse_and_r2)

sum = predict_q4_spending(id, monthly_transaction_data)
print(sum)



In [ ]:
wrld_transaction_data